<a href="https://colab.research.google.com/github/evillag/uncertainty_gan/blob/main/test_bench/TestBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mkdir: cannot create directory ‘/content/drive/MyDrive/cern/data/results’: File exists


In [1]:
IN_COLAB = True

try:
  import google.colab
  # Using Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

  !git clone https://github.com/evillag/uncertainty_gan.git
  !mv uncertainty_gan/mcd .
  !mv uncertainty_gan/feature_densities .
  !mv uncertainty_gan/test_bench .
  %rm -rf uncertainty_gan/

  !git clone https://gitlab.com/lambda-hse/lhcb-rich-gan-uncertainty.git
  !mv lhcb-rich-gan-uncertainty/experiments .
  !mv lhcb-rich-gan-uncertainty/src .
  %rm -rf lhcb-rich-gan-uncertainty/
  %rm -rf sample_data/
  %pip install tensorflow-addons

  # Dataset download and extraction
  !unzip -qq drive/MyDrive/cern/data/rich.zip

  # Model checkpoint download and extraction
  !unzip -qq drive/MyDrive/cern/data/checkpoints_dropout_0.01.zip

  # Model embeddings download and extraction
  !unzip -qq drive/MyDrive/cern/data/embeddings.zip

  # Results folder creation
  !mkdir /content/drive/MyDrive/cern/data/results

except:
  IN_COLAB = False

print(f'IN_COLAB: {IN_COLAB}')

Mounted at /content/drive
Cloning into 'uncertainty_gan'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 131 (delta 64), reused 52 (delta 19), pack-reused 0
Receiving objects: 100% (131/131), 17.34 MiB | 10.81 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Cloning into 'lhcb-rich-gan-uncertainty'...
remote: Enumerating objects: 210, done.
remote: Total 210 (delta 0), reused 0 (delta 0), pack-reused 210 (from 1)
Receiving objects: 100% (210/210), 2.94 MiB | 8.75 MiB/s, done.
Resolving deltas: 100% (94/94), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.5 MB/s eta 0:00:00
IN_COLAB: True


In [2]:
import numpy as np

from test_bench import get_checkpoint_name, load_particle_datasets, subsample_dataset
from test_bench.model import MonteCarloDropoutModel


# Test Bench for the Monte Carlo Dropout and Feature Density methods

1. Select sample data
2. Create a model
3. Generate a single target with single inference mode
4. Estimate MCD uncertainty
5. Estimate FD uncertainty

In [12]:
# Parameters
PARTICLE = 'pion'
CHECKPOINT_DP = 0.01
DROPOUT_TYPE = 'bernoulli_structured'
CHECKPOINT_BASE = 'checkpoints/'
DATA_DIR = 'rich/'
SUB_SAMPLE_PERCENT = 0.02

# MCD parameters
MCD_ENSEMBLE_SIZE = 300

# FD parameters
embeddings_dir = f'embeddings/'

# Save results path
output_dir = 'results/'
if IN_COLAB:
  output_dir = f'/content/drive/MyDrive/cern/data/{output_dir}'

# Load data and Sample selection

In [4]:
dataset = load_particle_datasets(PARTICLE, DATA_DIR)

Reading and concatenating datasets:
	rich/pion_-_down_2016_.csv
	rich/pion_+_up_2016_.csv
	rich/pion_-_up_2016_.csv
	rich/pion2_+_down_2016_.csv
	rich/pion2_-_up_2016_.csv
	rich/pion_+_down_2016_.csv
	rich/pion2_+_up_2016_.csv
	rich/pion2_-_down_2016_.csv
splitting to train/val/test
fitting the scaler
scaler train sample size: 2000000
scaler n_quantiles: 100000, time = 2.3753974437713623
scaling train set
scaling test set
converting dtype to <class 'numpy.float32'>
feats_train shape	(948527, 3)
targets_train shape	(948527, 5)
feats_val shape  	(526449, 3)
targets_val shape	(526449, 5)



In [5]:
# Draw a sample of the datasets
x_sample, y_sample = subsample_dataset(dataset['feats_val'], dataset['targets_val'], SUB_SAMPLE_PERCENT)
x_sample.shape, y_sample.shape

(TensorShape([10528, 3]), TensorShape([10528, 5]))

# Model creation

In [6]:
model = MonteCarloDropoutModel(
    PARTICLE,
    dropout_rate=CHECKPOINT_DP,
    checkpoint_dir=CHECKPOINT_BASE + get_checkpoint_name(PARTICLE, CHECKPOINT_DP, DROPOUT_TYPE),
    debug=True
)
generator = model.get_generator()

Generating model for pion with a dropout rate of 0.01
Layer 0
Layer 1
Layer 2
Layer 3
Layer 4

Generator:

Model: "virtual_ensemble_model"
________________________________________________________________________________________________
 Layer (type)                              Output Shape                          Param #        
 Inputs (InputLayer)                       [(None, 3)]                           0              
                                                                                                
 NoiseInjection (NoiseInjection)           (None, 67)                            0              
                                                                                                
 Layer_0/Dense (Dense)                     (None, 128)                           8704           
                                                                                                
 Layer_0/LeakyReLU (LeakyReLU)             (None, 128)                           0   

## Single model prediction

In [7]:
generator.single_model_inference_mode()
t_generated = generator.predict(x_sample)
t_generated

329/329 [==============================] - 5s 3ms/step


array([[ 0.34069768, -0.75628793,  0.6969006 , -0.26559168, -0.31229025],
       [-1.0982914 , -1.8875299 , -0.8781852 , -1.5892601 , -1.7735574 ],
       [ 0.5216234 ,  0.71218204,  0.3069851 , -0.73871994,  0.56137407],
       ...,
       [-0.8978682 , -1.5370419 , -0.8264393 , -1.3490703 , -1.496083  ],
       [ 0.6642503 , -1.191757  ,  1.1401606 , -0.587104  , -0.62596244],
       [ 0.05230185, -1.5115509 , -0.11086129, -0.85879713, -0.83539706]],
      dtype=float32)

In [17]:
# Save real and generated targets
np.save(output_dir + f'{PARTICLE}_t_generated.npy', t_generated)
np.save(output_dir + f'{PARTICLE}_y_real.npy', y_sample)

## Monte Carlo Dropout method

In [8]:
from mcd.MCDEvaluator import evaluate_model as mcd_evaluate_model

mcd_uncertainty, _ =  mcd_evaluate_model(model, x_sample, MCD_ENSEMBLE_SIZE)
mcd_uncertainty


Generating ensemble(300) predictions


100%|██████████| 300/300 [00:11<00:00, 25.55it/s]


<tf.Tensor: shape=(10528, 5), dtype=float32, numpy=
array([[0.4140107 , 0.44108438, 0.4775778 , 0.61591107, 0.79826605],
       [0.3021742 , 0.45253897, 0.6264502 , 0.45789215, 0.52805537],
       [0.13265656, 0.24616155, 0.09950522, 0.44376302, 0.8493591 ],
       ...,
       [0.34126005, 0.40928936, 0.57090616, 0.4934988 , 0.6186188 ],
       [0.4377561 , 0.44249243, 0.41206178, 0.6765101 , 0.95058113],
       [0.25326976, 0.43883586, 0.45758897, 0.52949965, 0.6475622 ]],
      dtype=float32)>

In [9]:
mcd_uncertainty.shape

TensorShape([10528, 5])

In [18]:
# Save MCD uncertainties
np.save(output_dir + f'{PARTICLE}_mcd_uncertainty.npy', mcd_uncertainty)

## Feature Densities method

### Uncertainty estimation

In [ ]:
from feature_densities.feature_density_evaluator import evaluate_model as fd_evaluate_model

train_embeddings = np.load(embeddings_dir + f'{PARTICLE}_train_embeddings.npy')
print(train_embeddings.shape)

fd_uncertainty_integration, _ = fd_evaluate_model(model, x_sample, known_embeddings=train_embeddings, likelihood_method='integration')

print('Feature Densities using INTEGRATION uncertainty score for x_sample:')
fd_uncertainty_integration

(948325, 128)
Generating an embeddings model
Fitting KDE functions to known embeddings
Calculating sample´s embeddings
329/329 [==============================] - 2s 3ms/step
Estimating sample´s feature densities


  0%|          | 2/10528 [00:26<37:32:37, 12.84s/it]

In [ ]:
# Save FD uncertainties with integration
np.save(output_dir + f'{PARTICLE}_fd_uncertainty_integration.npy', fd_uncertainty_integration)

In [ ]:
fd_uncertainty_normalized, _ = fd_evaluate_model(model, x_sample, known_embeddings=train_embeddings, likelihood_method='normalized')

print('Feature Densities using NORMALIZED uncertainty score for x_sample:')
fd_uncertainty_normalized

In [ ]:
# Save FD uncertainties normalized
np.save(output_dir + f'{PARTICLE}_fd_uncertainty_normalized.npy', fd_uncertainty_normalized)

### Generation of FD embeddings

In [ ]:
# from feature_densities.feature_density_evaluator import create_embeddings_model
# embeddings_model = create_embeddings_model(model)

In [ ]:
# train_embeddings, train_predictions = embeddings_model.predict(dataset['feats_train'])

In [ ]:
# test_embeddings, test_predictions = embeddings_model.predict(dataset['feats_val'])

In [ ]:
# !rm -r embeddings
# !mkdir embeddings

# np.save(embeddings_dir + f'{PARTICLE}_train_embeddings.npy', train_embeddings)
# np.save(embeddings_dir + f'{PARTICLE}_train_predictions.npy', train_predictions)
# np.save(embeddings_dir + f'{PARTICLE}_test_embeddings.npy', test_embeddings)
# np.save(embeddings_dir + f'{PARTICLE}_test_predictions.npy', test_predictions)